In [1]:
# Cell 1: Setup and imports
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import numpy as np
import pickle
from PIL import Image

import tensorflow as tf
from tensorflow.keras.models import load_model

import torch
from torchvision import transforms
from transformers import ViTForImageClassification

# Detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


2025-05-10 21:22:47.222396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746892367.244336   37842 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746892367.252206   37842 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-10 21:22:47.277236: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/mr/workspace/MajorProject/.venv_ensemble/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found

Using device: cuda


In [2]:
# Cell 2: Same model‐path variables as your notebook
k_saved_model_files = [
    "final models/FINAL/t2_breast_VGGAdam-epoch10.keras",
    "final models/FINAL/t3_breast_IRNAdam_NoPreProcess-epoch12.keras",
    "final models/FINAL/t3_breast_Xception_NoPreProcess-lrreduceP4-epoch02.keras",
    "final models/FINAL/t3_breast_ResNet50_NoPreProcess-epoch15.keras",
]
p_saved_model_files = [
    "final models/FINAL/t2_breast_vit_NoPreProcess_hdp0.2_adp0.2_lr1e-05-epoch4.pt"
]


In [3]:
# Cell 3: Preprocessing helpers
def preprocess_tf(image_path, target_size=(224,224)):
    img = Image.open(image_path).convert("RGB").resize(target_size)
    arr = np.array(img) / 255.0
    return arr

def preprocess_torch(image_path, target_size=(224,224)):
    tfm = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor(),
    ])
    img = Image.open(image_path).convert("RGB")
    return tfm(img).unsqueeze(0)


In [4]:
# Cell 4: Single‐image prediction functions (with prints)

def predict_single_image_keras(model_files, image_path, output_file):
    preds = []
    for mf in model_files:
        model = load_model(mf)
        x = preprocess_tf(image_path)
        p = model.predict(np.expand_dims(x, axis=0))[0]
        print(f"[Keras] Model '{os.path.basename(mf)}' prediction:", p)
        preds.append(p)
    with open(output_file, "wb") as f:
        pickle.dump({"model_files": model_files, "predictions": preds}, f)
    print(f"Saved Keras preds → {output_file}")

def predict_single_image_torch(model_file, image_path, output_file, device):
    # init ViT architecture
    model = ViTForImageClassification.from_pretrained(
        "google/vit-base-patch16-224-in21k", num_labels=2
    )
    sd = torch.load(model_file, map_location=device)
    model.load_state_dict(sd)
    model.to(device).eval()

    x = preprocess_torch(image_path).to(device)
    with torch.no_grad():
        out = model(x).logits.squeeze(0).cpu().numpy()
        probs = np.exp(out) / np.sum(np.exp(out))
    print(f"[PyTorch] Model '{os.path.basename(model_file)}' prediction:", probs)
    with open(output_file, "wb") as f:
        pickle.dump({"model_file": model_file, "predictions": probs}, f)
    print(f"Saved Torch preds → {output_file}")


In [11]:
# Cell 5: Ensemble voting utilities
def weighted_vote(preds, weights):
    return int(np.argmax(sum(p*w for p,w in zip(preds, weights))))

def majority_vote(preds):
    votes = [int(np.argmax(p)) for p in preds]
    return int(np.argmax(np.bincount(votes)))

def detailed_ensemble_from_files(keras_file, torch_file, keras_weights=None, torch_weight=None):
    import os, pickle

    # Load saved predictions
    kinfo = pickle.load(open(keras_file, "rb"))
    tinfo = pickle.load(open(torch_file, "rb"))

    # Prepare lists for preds and weights
    all_preds = kinfo["predictions"]
    weights = keras_weights if keras_weights else [1.0] * len(all_preds)
    all_preds.append(tinfo["predictions"])
    weights.append(torch_weight if torch_weight else 1.0)

    # Build a list of individual model outputs
    individual_outputs = []
    for mf, pred in zip(kinfo["model_files"], kinfo["predictions"]):
        individual_outputs.append({
            "model": os.path.basename(mf),
            "prediction": pred
        })
    individual_outputs.append({
        "model": os.path.basename(tinfo["model_file"]),
        "prediction": tinfo["predictions"]
    })

    # Compute ensemble votes
    weighted_pred = weighted_vote(all_preds, weights)
    majority_pred = majority_vote(all_preds)

    # Return everything
    return {
        "individual": individual_outputs,
        "weighted": weighted_pred,
        "majority": majority_pred
    }


In [12]:
# Cell 6: Run it on one image
img_path_A = "../breast_full_dataset/Malignant/20586934 (7).png"

img_path_B = "../breast_full_dataset/Benign/20586908 (10).png"



In [16]:
# 1) Generate and save individual preds
predict_single_image_keras(k_saved_model_files, img_path_A, "keras_preds.pkl")

I0000 00:00:1746892293.684267   36520 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4778 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5
I0000 00:00:1746892295.873549   37599 service.cc:148] XLA service 0x700c38003dc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746892295.873735   37599 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2025-05-10 21:21:35.891704: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746892296.004336   37599 cuda_dnn.cc:529] Loaded cuDNN version 90501
2025-05-10 21:21:37.311243: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[1,512,28,28]{3,2,1,0}, u8[0]{

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


I0000 00:00:1746892298.182135   37599 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[Keras] Model 't2_breast_VGGAdam-epoch10.keras' prediction: [0.9033662  0.09987863]
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
[Keras] Model 't3_breast_IRNAdam_NoPreProcess-epoch12.keras' prediction: [3.1579079e-04 9.9968195e-01]
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
[Keras] Model 't3_breast_Xception_NoPreProcess-lrreduceP4-epoch02.keras' prediction: [0.44705042 0.4073077 ]


2025-05-10 21:22:17.001462: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[1,128,28,28]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,128,28,28]{3,2,1,0}, f32[128,128,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-05-10 21:22:17.157792: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[1,256,14,14]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,256,14,14]{3,2,1,0}, f32[256,256,3,3]{3,2,1,0}, f32[256]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasAct

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
[Keras] Model 't3_breast_ResNet50_NoPreProcess-epoch15.keras' prediction: [0.00630238 0.9939294 ]
Saved Keras preds → keras_preds.pkl


In [7]:
predict_single_image_torch(p_saved_model_files[0], img_path_A, "torch_preds.pkl", device)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[PyTorch] Model 't2_breast_vit_NoPreProcess_hdp0.2_adp0.2_lr1e-05-epoch4.pt' prediction: [0.01692918 0.9830708 ]
Saved Torch preds → torch_preds.pkl


In [10]:
results = detailed_ensemble_from_files("keras_preds.pkl", "torch_preds.pkl", keras_acc, torch_acc)
print("Individual model outputs:")
for entry in results["individual"]:
    model_name = entry["model"]
    pred       = entry["prediction"]
    print(f"  • {model_name} → {pred}")

# Print the ensemble decisions
print("\nEnsemble results:")
print("  • Weighted vote →", results["weighted"])
print("  • Majority vote →", results["majority"])

Individual model outputs:
  • t2_breast_VGGAdam-epoch10.keras → [0.9033662  0.09987863]
  • t3_breast_IRNAdam_NoPreProcess-epoch12.keras → [3.1579079e-04 9.9968195e-01]
  • t3_breast_Xception_NoPreProcess-lrreduceP4-epoch02.keras → [0.44705042 0.4073077 ]
  • t3_breast_ResNet50_NoPreProcess-epoch15.keras → [0.00630238 0.9939294 ]
  • t2_breast_vit_NoPreProcess_hdp0.2_adp0.2_lr1e-05-epoch4.pt → [0.01692918 0.9830708 ]

Ensemble results:
  • Weighted vote → 1
  • Majority vote → 1
